In [1]:
from pprint import pp
import datetime
import argparse
from pathlib import Path
import sqlite3
import traceback
import json
import re
from track_records.data_man import *
from prettytable import PrettyTable

def adapt_date_iso(val):
    """Adapt datetime.date to ISO 8601 date."""
    return val.isoformat()
sqlite3.register_adapter(datetime.date, adapt_date_iso)
def convert_date(val):
    """Convert ISO 8601 date to datetime.date object."""
    return datetime.date.fromisoformat(val)
sqlite3.register_converter('date', convert_date)


In [2]:
results_2024 = execute_named_query('data/track_records.sqlite',
    "data/queries.sql",
    "get_all_athletes_from_team_in_year<>", ("Saint Joseph Catholic School", "2024"))
results_2025 = execute_named_query('data/track_records.sqlite',
    "data/queries.sql",
    "get_all_athletes_from_team_in_year<>", ("Saint Joseph Catholic School", "2025"))

names_2025 = [athlete['athlete_name'] for athlete in results_2025]
names_2024 = [athlete['athlete_name'] for athlete in results_2024]
# pp(names_2024)
# pp(names_2025)

# merge the names list
names_2024_2025 = list(set(names_2024 + names_2025))
pp(names_2024_2025)

['',
 'Emerson Harres',
 'Anja Scherpa',
 'Saint Joseph Catholic School f4x200m relay team',
 'Patrick Lamb',
 'Elijah Ruybal',
 'Elizabeth Davis',
 'Evelyn Davis',
 'Sienna Harden',
 'Mason Bauer',
 'Julia Alder',
 'Emily Vignale',
 'Colton Harres',
 'Sylvie Dellenbach',
 'Jude Cummins',
 'Felicity Williams',
 'Joey Peacock',
 'Faith Williams',
 'Abbey Rojeski',
 'Vivian Bozek',
 'Daniel Antinora',
 'Isaac Watkins',
 'James Balk',
 'Paige Carlson',
 'Crosby Orinoz',
 'Howard Valdiviez',
 'Morgan Carlson',
 'Cora Bauer',
 'Marilyn Wade',
 'Zoe Moran',
 'Evelyn Mast',
 'John Paul Bradley',
 'Lucy Mast',
 'Saint Joseph Catholic School m4x100m relay team',
 'Saint Joseph Catholic School f4x100m relay team']


Set the names I want to use.

In [3]:
# names = [
#  'Morgan Carlson',
#  'Cora Bauer',
#  'Paige Carlson',
#  'Felicity Williams',
#  'Abbey Rojeski',
#  'Emerson Harres',
#  'Marilyn Wade',
#  'Vivian Bozek',
#  'Lucy Mast',
#  'Daniel Antinora',
#  'Elijah Ruybal',
#  'Sienna Harden',
#  'Joey Peacock',
#  'Evelyn Mast']

In [4]:
athlete_records = execute_named_query('data/track_records.sqlite',
    "data/queries.sql",
    "get_records_from_all_athletes_on_team<>_in_year<>", ("Saint Joseph Catholic School", "2024"))
    # pp(athlete_records)

print(athlete_records)

[{'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls 100 meter dash', 'meet_date': '2025-05-03', 'location': '1', 'result': '15.13', 'result_sort': 15.13}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls 100 meter high hurdles', 'meet_date': '2024-04-26', 'location': '1', 'result': '22.99', 'result_sort': 22.99}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls 200 meter dash', 'meet_date': '2025-05-10', 'location': '1', 'result': '32.04', 'result_sort': 32.04}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls discus', 'meet_date': '2025-05-03', 'location': '1', 'result': '67-7.5', 'result_sort': -67.58749999999999}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls long jump', 'meet_date': '2025-04-12', 'location': '22', 'result': '12-11.0', 'result_sort': -12.916666666666666}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls shot put', 'meet_date': '2024-04-22', 'location': '1', 'result': '21-10.75', 'result_sort': -21.895833333333332}, {'athlete_name

Get the athletes for the names above.

In [5]:
placeholders = f"({', '.join(['?'] * len(names_2025))})"
# print('placeholders = ', placeholders)
athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_personal_records_for_athletes<>", tuple(names_2025), PLACEHOLDER1=placeholders
)
# Create a PrettyTable object with the specified headings
table = PrettyTable(['Athlete Name', 'Event Name', 'Meet Date', 'Result', 'Location'])

# Add rows to the table
for record in athlete_records:
    table.add_row([record['athlete_name'], record['event_name'], record['meet_date'], record['result'], record['location']])

# Print the table
# print(table)


Print out the report.

In [6]:
report = PDFReport("track_records_report.pdf")
report.create_pdf(athlete_records)

In [7]:
placeholders = f"({', '.join(['?'] * len(names_2024_2025))})"
# print('placeholders = ', placeholders)
athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_results_from_athletes<>", tuple(names_2024_2025) + ("Saint Joseph Catholic School",), PLACEHOLDER1=placeholders
)

# Create a dictionary to store the data in the desired format
formatted_data = {}

# Process each record to organize data by athlete and event
for record in athlete_records:
    athlete = record['athlete_name']
    event = record['event_name']
    date = record['meet_date']
    result = record['result']

    # Initialize nested dictionaries if not already present
    if athlete not in formatted_data:
        formatted_data[athlete] = {}
    if event not in formatted_data[athlete]:
        formatted_data[athlete][event] = {}

    # Assign the result to the corresponding date column
    formatted_data[athlete][event][date] = result

# Create a PrettyTable object with dynamic date columns
dates = sorted({record['meet_date'] for record in athlete_records})
columns = ['Athlete Name', 'Event Name'] + dates
table = PrettyTable(columns)

# Populate the table with the formatted data
for athlete, events in formatted_data.items():
    for event, results in events.items():
        row = [athlete, event] + [results.get(date, '') for date in dates]
        table.add_row(row)

# Print the table
# print(table)

In [8]:
placeholders = f"({', '.join(['?'] * len(names_2025))})"
# print('placeholders = ', placeholders)
athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_results_from_athletes<>", tuple(names_2025) + ("Saint Joseph Catholic School",), PLACEHOLDER1=placeholders
)


# Generate a PDF report for the results
report = PDFReport("athlete_results.pdf")
report.create_results_pdf(athlete_records)

In [15]:
placeholders = f"({', '.join(['?'] * len(names_2025))})"
# print('placeholders = ', placeholders)
this_date = "2025-05-10"
athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_prs_from_athletes_on_date<>", tuple(names_2025 +  ["Saint Joseph Catholic School"]
                                              + [this_date]), PLACEHOLDER1=placeholders
)

# table_str = "| Athlete Name | Event Name | Result |\n"
# table_str += "|---|---|---|\n"
# for record in athlete_records:
#     table_str += f"| {record['athlete_name']} | {record['event_name']} | {record['result']} |\n"
# print(table_str)

# Create a PrettyTable object with the specified headings
table = PrettyTable(['Athlete Name', 'Event Name', 'Result'])

# Add rows to the table
for record in athlete_records:
    table.add_row([record['athlete_name'], record['event_name'], record['result']])

# Print the table
print(table)

# Generate a PDF report for the results
report = PDFReport("athlete_PRs_on_2025-05-10.pdf")
report.create_results_pdf(athlete_records, "PRs on " + this_date)

+-------------------------------------------------+------------------------------+---------+
|                   Athlete Name                  |          Event Name          |  Result |
+-------------------------------------------------+------------------------------+---------+
|                  Abbey Rojeski                  |     Girls 200 meter dash     |  32.04  |
|                  Colton Harres                  |     Boys 400 meter dash      | 1:19.82 |
|                  Colton Harres                  |        Boys shot put         | 13-7.25 |
|                 Daniel Antinora                 |     Boys 200 meter dash      |  35.38  |
|                 Daniel Antinora                 |    Boys 200 meter hurdles    |  37.30  |
|                 Daniel Antinora                 |        Boys long jump        | 11-4.25 |
|                  Elijah Ruybal                  |        Boys shot put         |   21-1  |
|                  Emerson Harres                 | Girls 100 meter hi

In [10]:
athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_top_5_places_from_team_on_date<>", ("Saint Joseph Catholic School", this_date)
)

# Sort the records by athlete name
athlete_records_sorted = sorted(athlete_records, key=lambda x: x['athlete_name'])

table_str = "| Athlete Name | Event Name | Result | Place |\n"
table_str += "|---|---|---|---|\n"
for record in athlete_records_sorted:
    table_str += f"| {record['athlete_name']} | {record['event_name']} | {record['result']} | {record['place']} |\n"
print(table_str)

| Athlete Name | Event Name | Result | Place |
|---|---|---|---|
| Abbey Rojeski | Girls discus | 63-3 | 5 |
| Abbey Rojeski | Girls long jump | 12-6 | 3 |
| Emerson Harres | Girls 100 meter high hurdles | 20.34 | 5 |
| Evelyn Mast | Girls discus | 63-3.75 | 4 |
| Evelyn Mast | Girls shot put | 27-11.5 | 3 |
| Felicity Williams | Girls 200 meter hurdles | 35.27 | 1 |
| Lucy Mast | Girls long jump | 12-1 | 5 |
| Marilyn Wade | Girls 1600 meter run | 5:49.00 | 1 |
| Marilyn Wade | Girls 400 meter dash | 1:06.68 | 2 |
| Marilyn Wade | Girls 800 meter run | 2:37.31 | 1 |
| Marilyn Wade | Girls discus | 77-7 | 1 |
| Morgan Carlson | Girls 100 meter high hurdles | 19.60 | 3 |
| Morgan Carlson | Girls shot put | 27-6 | 4 |
| Saint Joseph Catholic School f4x100m relay team | Girls 4x100 meter relay | 59.04 | 2 |
| Saint Joseph Catholic School f4x200m relay team | Girls 4x200 meter relay | 2:11.73 | 4 |
| Vivian Bozek | Girls 100 meter high hurdles | 19.51 | 2 |
| Zoe Moran | Girls 200 meter hu

In [11]:
athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_top_8_athletes_per_event_for_team_in_year<>", ("Saint Joseph Catholic School", "2025")
)

# Create a dictionary to store the data in the desired format
formatted_data = {}

# Process each record to organize data by event and athlete
for record in athlete_records:
    event = record['event_name']
    athlete = record['athlete_name']
    result = record['result']

    # Initialize nested dictionaries if not already present
    if event not in formatted_data:
        formatted_data[event] = []

    formatted_data[event].append({'athlete': athlete, 'result': result})

# Create a PrettyTable object for each event
tables = {}
for event, records in formatted_data.items():
    table = PrettyTable(['Athlete Name', 'Result'])
    for record in records:
        table.add_row([record['athlete'], record['result']])
    tables[event] = table

# Print the tables
for event, table in tables.items():
    print(f"Event: {event}")
    print(table)
    print("\n")
#pp(athlete_records)


Event: Boys 100 meter dash
+-------------------+--------+
|    Athlete Name   | Result |
+-------------------+--------+
|    Joey Peacock   | 14.81  |
|    Mason Bauer    |  15.2  |
|     James Balk    | 15.66  |
|  Daniel Antinora  | 16.02  |
|  Howard Valdiviez | 17.54  |
| John Paul Bradley | 17.87  |
|   Isaac Watkins   | 21.91  |
+-------------------+--------+


Event: Boys 1600 meter run
+---------------+---------+
|  Athlete Name |  Result |
+---------------+---------+
| Elijah Ruybal | 7:00.43 |
+---------------+---------+


Event: Boys 200 meter dash
+------------------+--------+
|   Athlete Name   | Result |
+------------------+--------+
|   Joey Peacock   | 31.55  |
| Daniel Antinora  | 35.38  |
|  Colton Harres   | 37.97  |
| Howard Valdiviez | 40.39  |
+------------------+--------+


Event: Boys 200 meter hurdles
+-----------------+--------+
|   Athlete Name  | Result |
+-----------------+--------+
|    James Balk   | 35.84  |
|   Mason Bauer   | 36.37  |
| Daniel Antinora

In [14]:
from prettytable import PrettyTable

athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_top_8_finishes_from_team_on_date<>", ("Saint Joseph Catholic School", "2025-05-10")
)

#pp(athlete_records)
# Sort the records by athlete name

athlete_records_sorted = sorted(athlete_records, key=lambda x: x['place'])

# Create a PrettyTable object with the specified headings
table = PrettyTable(['Place', 'Athlete Name', 'Event Name', 'Result'])

# Add rows to the table
for record in athlete_records_sorted:
    table.add_row([record['place'], record['athlete_name'], record['event_name'], record['result']])

# Print the table
print(table)


+-------+-------------------------------------------------+------------------------------+---------+
| Place |                   Athlete Name                  |          Event Name          |  Result |
+-------+-------------------------------------------------+------------------------------+---------+
|   1   |                   Marilyn Wade                  |     Girls 1600 meter run     | 5:49.00 |
|   1   |                Felicity Williams                |   Girls 200 meter hurdles    |  35.27  |
|   1   |                   Marilyn Wade                  |     Girls 800 meter run      | 2:37.31 |
|   1   |                   Marilyn Wade                  |         Girls discus         |   77-7  |
|   2   |                   Vivian Bozek                  | Girls 100 meter high hurdles |  19.51  |
|   2   |                   Marilyn Wade                  |     Girls 400 meter dash     | 1:06.68 |
|   2   | Saint Joseph Catholic School f4x100m relay team |   Girls 4x100 meter relay    | 